In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

In [2]:
# Dates have to extracted
# data = pd.read_csv('train.csv', parse_dates=['Dates'])
data = pd.read_csv('./dataset/train.csv', parse_dates=['Dates'])
test = pd.read_csv('./dataset/test.csv', parse_dates=['Dates'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867873 entries, 0 to 867872
Data columns (total 10 columns):
Dates         867873 non-null datetime64[ns]
Category      867873 non-null object
Descript      867873 non-null object
DayOfWeek     867873 non-null object
PdDistrict    867873 non-null object
Resolution    867873 non-null object
Address       867873 non-null object
X             867873 non-null float64
Y             867873 non-null float64
Id            867873 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 66.2+ MB


In [4]:
# No null values in the data-frame
data.isnull().values.any()

False

In [5]:
# Dates
data_week_dict = {
    'Monday': 1,
    'Tuesday':2,
    'Wednesday':3,
    'Thursday':4,
    'Friday':5,
    'Saturday':6,
    'Sunday':7
}

data['Hour'] = data.Dates.dt.hour
data['Minutes'] = data.Dates.dt.minute
data['Year'] = data.Dates.dt.year
data['Month'] = data.Dates.dt.month
data['Day'] = data.Dates.dt.day
data['DayOfWeekNum'] = data['DayOfWeek'].replace(data_week_dict)

test['Hour'] = test.Dates.dt.hour
test['Minutes'] = test.Dates.dt.minute
test['Year'] = test.Dates.dt.year
test['Month'] = test.Dates.dt.month
test['Day'] = test.Dates.dt.day
test['DayOfWeekNum'] = test['DayOfWeek'].replace(data_week_dict)

In [6]:
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Hour,Minutes,Year,Month,Day,DayOfWeekNum
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,17,40,2013,6,28,5
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2,46,2004,2,19,4
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,0,1,2007,11,14,3
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,18,30,2007,12,27,4
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,17,2,2012,9,9,7


In [7]:
labelencoder = LabelEncoder()

In [8]:
data['ResolutionNum'] = labelencoder.fit_transform(data['Resolution'])
data['PdDistrictNum'] = labelencoder.fit_transform(data['PdDistrict'])
data['CategoryNum'] = labelencoder.fit_transform(data['Category'])

test['ResolutionNum'] = labelencoder.fit_transform(test['Resolution'])
test['PdDistrictNum'] = labelencoder.fit_transform(test['PdDistrict'])

In [9]:
data = data[data.X < -121]
data = data[data.Y < 40]

test = test[test.X < -121]
test = test[test.Y < 40]

In [10]:
def getCapsAddress(i):
    s=''
    for j in i.split():
        if(j.isupper()):
            s=s+' '+j
    return s[1:]

data['newAddress'] = data.Address.apply(lambda a:getCapsAddress(a))
test['newAddress'] = test.Address.apply(lambda a:getCapsAddress(a))

In [11]:
data['newAddressNum'] = labelencoder.fit_transform(data.newAddress)
test['newAddressNum'] = labelencoder.fit_transform(test.newAddress)

In [12]:
data['Address_CrossRoad'] = data['Address'].str.contains('/')
test['Address_CrossRoad'] = test['Address'].str.contains('/')

topN_address_list = data['Address'].value_counts()
topN_address_list = topN_address_list[topN_address_list >=100]
topN_address_list = topN_address_list.index
print(topN_address_list)

data['Address_clean'] = data['Address']
test['Address_clean'] = test['Address']
data.loc[~data['Address'].isin(topN_address_list), 'Address_clean'] = 'Others'
test.loc[~test['Address'].isin(topN_address_list), 'Address_clean'] = 'Others'
print(data.shape)

crossload = data[data['Address_clean'].str.contains('/')]
crossroad_list = crossload['Address_clean'].unique()
print(len(crossroad_list))

Index(['800 Block of BRYANT ST', '800 Block of MARKET ST',
       '2000 Block of MISSION ST', '1000 Block of POTRERO AV',
       '900 Block of MARKET ST', '0 Block of TURK ST', '0 Block of 6TH ST',
       '300 Block of ELLIS ST', '400 Block of ELLIS ST',
       '16TH ST / MISSION ST',
       ...
       'POLK ST / HEMLOCK ST', '700 Block of GONZALEZ DR',
       '300 Block of 8TH ST', '900 Block of GRANT AV', 'MARKET ST / HYDE ST',
       '500 Block of JACKSON ST', '800 Block of MISSOURI ST',
       '14TH ST / FOLSOM ST', '600 Block of ALABAMA ST',
       '1000 Block of GRANT AV'],
      dtype='object', length=1616)
(867806, 23)
475


In [13]:
for address in crossroad_list:
    address_split = address.split('/')
    reverse_address = address_split[1].strip() + ' / ' + address_split[0].strip()
    data.loc[data['Address_clean'] == reverse_address, 'Address_clean'] = address
    test.loc[test['Address_clean'] == reverse_address, 'Address_clean'] = address
crossload = data[data['Address_clean'].str.contains('/')]
crossroad_list = crossload['Address_clean'].unique()
print(len(crossroad_list))

le = LabelEncoder()
data['Address_clean_encode'] = le.fit_transform(data['Address_clean'])
print(data.shape)

321
(867806, 24)


In [14]:
le = LabelEncoder()
test['Address_clean_encode'] = le.fit_transform(test['Address_clean'])

In [15]:
corr = data.corr()
print(corr['CategoryNum'].sort_values(ascending=False))

CategoryNum             1.000000
Address_clean_encode    0.070796
Address_CrossRoad       0.069896
ResolutionNum           0.039676
Hour                    0.023701
Id                      0.016881
DayOfWeekNum            0.000537
Day                     0.000354
Month                  -0.000360
Year                   -0.016795
newAddressNum          -0.019639
Minutes                -0.021945
X                      -0.030525
PdDistrictNum          -0.039477
Y                      -0.058963
Name: CategoryNum, dtype: float64


In [16]:
# data.drop('Dates', axis=1, inplace=True)
# data.drop('Descript', axis=1, inplace=True)
# data.drop('DayOfWeek', axis=1, inplace=True)
# data.drop('PdDistrict', axis=1, inplace=True)
# data.drop('Resolution', axis=1, inplace=True)
# data.drop('Address', axis=1, inplace=True)
# data.drop('Street', axis=1, inplace=True)
# data.drop('Category', axis=1, inplace=True)
# data.drop('Minutes', axis=1, inplace=True)
# data.drop('newAddress', axis=1, inplace=True)
# data.drop('Id',axis=1,inplace=True)

In [17]:
features=['X','Y','Hour','Minutes','Year','Month','Day','DayOfWeekNum', 'PdDistrictNum',
          'Address_CrossRoad', 'Address_clean_encode']

In [ ]:
# for i in data.CategoryNum.unique():
#     print(i,labelencoder.inverse_transform(data.CategoryNum.unique())[i])
#     data[data.CategoryNum==i].hist(bins=50, figsize=(20,15))
#     plt.show()
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Random seed has been set - As per the guidlines of the competition
train, test = train_test_split(data, test_size=0.3, random_state=3, shuffle=True)

In [ ]:
ytrain_ = train['CategoryNum']
Xtrain_ = train[features]
ytest_ = test['CategoryNum']
Xtest_ = test[features]

In [18]:
y_train = data['CategoryNum']
X_train = data[features]
# y_test = test['CategoryNum']
X_test = test[features]

In [19]:
X_test.shape

(10000, 11)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',max_iter=1000)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_test)
log_loss(y_test,pred)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(Xtrain_,ytrain_)
print(clf.score(Xtest_,ytest_))
pred = clf.predict_proba(Xtest_)
print(log_loss(ytest_,pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_state = 42
max_depth = 7
min_weight_fraction_leaf = 1
n_estimators = 500
n_jobs = -1

model = RandomForestClassifier(
    max_depth=max_depth,
    n_estimators=n_estimators,
    n_jobs=n_jobs,
    min_weight_fraction_leaf=min_weight_fraction_leaf
)

# model.fit(Xtrain_,ytrain_)
# print(model.score(Xtest_,ytest_))
# pred = model.predict_proba(Xtest_)
# print(log_loss(ytest_,pred))

In [20]:
import xgboost as xgb

seed = 42
max_depth = 9
learning_rate = 0.2
min_child_weight = 1.5
n_estimators = 100

model = xgb.XGBClassifier(
    objective='multi:softprob', 
    seed=seed, 
    max_depth=max_depth,
    nthread=8,
    n_jobs=-1,
    min_child_weight=min_child_weight,
    learning_rate=learning_rate,
    n_estimators = n_estimators
)

In [21]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=9, min_child_weight=1.5, missing=None, n_estimators=100,
       n_jobs=-1, nthread=8, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [ ]:
score = -1 * cross_val_score(model, Xtrain_, ytrain_, scoring='neg_log_loss', cv=5, n_jobs=-1)

In [ ]:
print("Score = {0:.5f}".format(score.mean()))

In [ ]:
score

In [22]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=9, min_child_weight=1.5, missing=None, n_estimators=100,
       n_jobs=-1, nthread=8, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [23]:
import pickle

In [24]:
pickle.dump(model, open("xgboost_wo_res51118.p", "wb"))

In [25]:
model = pickle.load(open("xgboost_wo_res51118.p", "rb"))
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=9, min_child_weight=1.5, missing=nan, n_estimators=100,
       n_jobs=-1, nthread=8, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [26]:
predictions = model.predict_proba(X_test)

In [27]:
submission = pd.DataFrame(predictions)
submission.columns = sorted(data.Category.unique())
submission['Id'] = test['Id']
submission

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS,Id
0,0.000260,0.067840,0.000121,0.000070,0.046819,0.001096,0.000272,0.007412,0.002365,0.000157,...,0.018379,0.001708,0.000095,0.015313,0.006512,0.013360,0.012693,0.013963,0.004008,349598
1,0.003989,0.181338,0.000087,0.000078,0.047200,0.007282,0.001964,0.028057,0.009951,0.000598,...,0.002713,0.006447,0.000640,0.035744,0.013108,0.047563,0.011870,0.083454,0.015491,766313
2,0.000645,0.029035,0.000011,0.000079,0.001337,0.002279,0.016321,0.017434,0.010276,0.000032,...,0.000855,0.005633,0.000024,0.033303,0.000130,0.019659,0.003378,0.049466,0.013960,169887
3,0.000129,0.067917,0.000030,0.000086,0.000593,0.001422,0.003744,0.016184,0.002619,0.000199,...,0.003007,0.001368,0.000087,0.009571,0.000134,0.068388,0.075917,0.012950,0.003959,594704
4,0.000372,0.050189,0.000016,0.000197,0.000626,0.002231,0.000698,0.009742,0.016120,0.000026,...,0.001094,0.004897,0.000030,0.023967,0.000678,0.028347,0.027468,0.054489,0.009226,47900
5,0.000061,0.107591,0.000027,0.000185,0.001014,0.001225,0.000578,0.085995,0.004421,0.000094,...,0.005879,0.001649,0.000022,0.021110,0.000197,0.041580,0.019801,0.018496,0.002120,339260
6,0.000645,0.060329,0.000036,0.000168,0.115526,0.001207,0.000051,0.010958,0.001443,0.000286,...,0.012472,0.002320,0.000075,0.015245,0.004107,0.094075,0.074328,0.010755,0.001723,169575
7,0.000401,0.014304,0.000033,0.000050,0.000225,0.000831,0.000061,0.002172,0.000496,0.000023,...,0.001356,0.000780,0.000020,0.013659,0.000039,0.024795,0.135616,0.002419,0.001853,681483
8,0.000682,0.059394,0.001666,0.000298,0.061596,0.001406,0.000073,0.005623,0.000754,0.005114,...,0.006894,0.003429,0.000345,0.046527,0.007584,0.040895,0.023270,0.010652,0.003714,276938
9,0.000452,0.090722,0.000041,0.000464,0.075473,0.000398,0.000233,0.003390,0.001512,0.000514,...,0.005092,0.003461,0.000267,0.041674,0.004393,0.079695,0.049424,0.008295,0.005298,146369


In [28]:
submission.to_csv('submission.csv', index=False)

TODO
- Pick relevant/independent columns and pass to the classifier
- Normalize the data
- Perform Exploratory Data Analysis, Visualization
- Finalize the feature list
- Build the models
- Voila!